In [1]:
import sys, os
sys.path.append("/cs/home/kf39/PycharmProjects/Spark/")
from src.spark import Spark
import src.tweet_volume as funcs
import matplotlib.pyplot as plt
from pyspark.sql import functions as fs
import os 
import pandas as pd
from src.plotting import double_plot
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import src.nlp.clean as clean


In [2]:
spark = Spark('load', 'local')
sess = spark.session()

In [3]:
df = funcs.load_dataframe(sess, '/cs/home/kf39/PycharmProjects/Spark/data/*.json', funcs.schema)
df2 = funcs.parse_timestmap(df)

In [4]:
eth_df = df2.filter(fs.lower(df['text']).like("%ether%") | fs.lower(df['text']).like("%eth%") | fs.lower(df['text']).like("%ethereum%"))


In [5]:

eth_clean = clean.clean_tweets(eth_df, "text")
eth_train, eth_test, eth_eval = eth_clean.randomSplit([0.98, 0.1, 0.1])

In [6]:
eth_train.first().text

u' #ethereum #eth'

In [7]:
from sparknlp.pretrained.pipeline.en import SentimentPipeline

In [8]:
sentimentPipeline = SentimentPipeline().pretrained()

Py4JJavaError: An error occurred while calling z:com.johnsnowlabs.nlp.pretrained.PythonResourceDownloader.downloadPipeline.
: java.lang.NoClassDefFoundError: com/typesafe/config/ConfigMergeable
	at com.johnsnowlabs.util.ConfigHelper$.getConfigValue(ConfigHelper.scala:8)
	at com.johnsnowlabs.util.ConfigHelper$.getConfigValueOrElse(ConfigHelper.scala:15)
	at com.johnsnowlabs.nlp.pretrained.ResourceDownloader$.<init>(ResourceDownloader.scala:44)
	at com.johnsnowlabs.nlp.pretrained.ResourceDownloader$.<clinit>(ResourceDownloader.scala)
	at com.johnsnowlabs.nlp.pretrained.PythonResourceDownloader$.downloadPipeline(ResourceDownloader.scala:137)
	at com.johnsnowlabs.nlp.pretrained.PythonResourceDownloader.downloadPipeline(ResourceDownloader.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang

In [7]:
tokeniser = Tokenizer(inputCol="text", outputCol="words")
hashtf = HashingTF(numFeatures=2**12, inputCol="words", outputCol="tf")
idf = IDF(inputCol="tf", outputCol="features", minDocFreq=5)
string_index = StringIndexer(inputCol="target", outputCol="label")
pipeline = Pipeline(stages=[tokeniser, hashtf, idf, string_index])

# Sentiment feature selection pipeline
eth_model = pipeline.fit(eth_train)
eth_train_df = eth_model.transform(eth_train)
eth_eval_df = eth_model.transform(eth_eval)

# Fit a classifier to classify as either positive or negative sentiment
lr = LogisticRegression(maxIter=100)
classifier = lr.fit(eth_train_df)
predictions = classifier.transform(eth_eval_df)

report = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
bin_rep = report.evaluate(predictions)
accuracy = predictions.filter(predictions.label == predictions.prediction).count() / float(eth_eval_df.count())

print("RocAuc: %.4f, Accuracy: %.4f" % (bin_rep, accuracy))

IllegalArgumentException: u'Field "target" does not exist.'